In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset
from torchvision.transforms import functional as VF
from torchmetrics.classification import BinaryJaccardIndex
from torchvision import models, datasets, tv_tensors
from torchvision.transforms import v2
import pandas as pd
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from tqdm import tqdm
import random
import data_pipeline

In [2]:
import unet
import baseline
import json
model = unet.UNet(n_channels=1, n_classes=1)
# model = baseline.Baseline()
model.load_state_dict(torch.load('./unet_5.pt', map_location=torch.device('cpu')))
test_df = data_pipeline.build_test_dataframe(use_processed_images=True)
train_df = data_pipeline.build_dataframe(use_processed_images=True)

CUDA not available, switching to MPS


In [3]:
X_test, X_names, X_train = data_pipeline.build_test_dataloaders(test_df, train_df)

In [4]:
test_dl = DataLoader(TensorDataset(X_test), batch_size=1)

In [5]:
predictions = {}
model.eval()
for index, x in tqdm(enumerate(test_dl)):
  out = model(x[0])
  preds = (F.sigmoid(out) > .5)*1.
  name = X_names[index][0]
  predictions[name] = preds.flatten().tolist()

0it [00:00, ?it/s]

2538it [00:25, 98.06it/s] 


In [6]:
preds_df = pd.DataFrame.from_dict(predictions, orient='index')

In [8]:
preds_df.to_csv('KIRBY_predictions_5.csv')

# Pipe Prediction

In [9]:
def get_pipe_id_dataloaders(train_dataframe, valid_dataframe, p):
  data = torch.from_numpy(np.vstack(train_dataframe['data'].to_numpy()))
  data = torch.nan_to_num(data)
  labels = torch.from_numpy(np.vstack(train_dataframe['labels'].to_numpy()))

  valid_data = torch.from_numpy(np.vstack(valid_dataframe['data'].to_numpy()))
  valid_data = torch.nan_to_num(valid_data)
  valid_labels = torch.from_numpy(np.vstack(valid_dataframe['well_number'].to_numpy())).squeeze() - 1
  X_names = np.vstack(valid_dataframe['filename'].to_numpy())

  data = data[p]
  offset = int(len(data) * .5)

  X_train, X_valid = data[:offset], valid_data
  Y_train, Y_valid = labels[:offset], valid_labels

  scaler = RobustScaler()
  scaler.fit(X_train)
  X_train = torch.tensor(scaler.transform(X_train)).float().reshape(-1, 1, 36, 36)
  X_valid = torch.tensor(scaler.transform(X_valid)).float().reshape(-1, 1, 36, 36)

  return X_valid, X_train, Y_train, X_names


In [10]:
import pipe_identifier
import json
with open('./classification_train_set_permutation.json') as f:
  p = json.load(f)
X_test, X_train, Y_train, X_names = get_pipe_id_dataloaders(build_train_dataframe(), build_test_dataframe(), p)
model = pipe_identifier.PipeIdentifier(num_classes=15)
model.load_state_dict(torch.load('./well_classifier_1.pt', map_location=torch.device('cpu')))
pipe_id_dataloader = DataLoader(TensorDataset(X_test), batch_size=1)

In [27]:
predictions = {}
model.eval()
for index, x in tqdm(enumerate(pipe_id_dataloader)):
  out = model(x[0])
  preds = torch.argmax(F.softmax(out, dim=1))
  name = X_names[index][0]
  predictions[name] = preds.flatten().tolist()

2538it [00:17, 148.96it/s]


In [28]:
preds_df = pd.DataFrame.from_dict(predictions, orient='index')

In [29]:
preds_df.value_counts()

2     2377
8      137
9       16
10       7
14       1
Name: count, dtype: int64

In [21]:
predictions = {}
model.eval()
for index, x in tqdm(enumerate(X_train)):
  out = model(x.unsqueeze(dim=0))
  preds = torch.argmax(F.softmax(out, dim=1))
  name = Y_train[index][0]
  predictions[name] = preds.flatten().tolist()

4717it [00:31, 150.14it/s]


In [22]:
preds_df = pd.DataFrame.from_dict(predictions, orient='index')